In [5]:
from pathlib import Path
from collections import defaultdict
from itertools import combinations
import dask.dataframe as dd
import dask
import pandas as pd

In [6]:
out_path = Path("../../Molecular_database/HAC_9")
pa = list(out_path.glob("*.parquet"))
clas = defaultdict(list)
for p in pa:
    db = p.stem.split("_")[1].split("_")[0].strip("db")
    clas[db].append(p)
comb = list(combinations(clas, 2))

In [7]:
clas.keys()

dict_keys(['003', '012', '004', '008', '007', '011', '005', '009'])

In [ ]:
from dask.distributed import Client
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 31.08 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39583,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:36171,Total threads: 2
Dashboard: http://127.0.0.1:36133/status,Memory: 7.77 GiB
Nanny: tcp://127.0.0.1:40669,


2025-11-18 13:24:49,257 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 0862a5d796b33c9cd7ded5e3e58f3499 initialized by task ('shuffle-transfer-0862a5d796b33c9cd7ded5e3e58f3499', 599) executed on worker tcp://127.0.0.1:43029
2025-11-18 13:24:54,783 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 3d002b6e626f8a61bbe22e0222a99170 initialized by task ('shuffle-transfer-3d002b6e626f8a61bbe22e0222a99170', 75) executed on worker tcp://127.0.0.1:35347
2025-11-18 13:24:59,165 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 0862a5d796b33c9cd7ded5e3e58f3499 deactivated due to stimulus 'task-finished-1763468699.1507285'
2025-11-18 13:24:59,846 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 3c6dfaedc06fe3546d96da9f94e3c0f2 initialized by task ('shuffle-transfer-3c6dfaedc06fe3546d96da9f94e3c0f2', 9) executed on worker tcp://127.0.0.1:36171
2025-11-18 13:25:00,658 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 7a6ca6832450f4f8daa350f346b7

In [9]:
dedup_dfs = {}

lazy_results = []
previous = []
smiles_col="SMILES"
# Build all lazy computations first
for db_id, path in clas.items():
    print(db_id)
    df = dd.read_parquet(path, columns=[smiles_col])
    
    df_dedup = df.drop_duplicates(subset=[smiles_col])
    unique = df_dedup.map_partitions(len).sum()
    dedup_dfs[db_id] = df_dedup
    # Collect both lazy results for single batch compute
    lazy_results.append(unique)
    
# Compute all totals and uniques at once
computed_values = dask.compute(*lazy_results)

# Assign results back in the same order
counts = dict(zip(clas.keys(), computed_values))

003
012
004
008
007
011
005
009


In [10]:
def modify_counts(counts):
    names = {}
    n = []
    for x in counts:
        if len(x.split("_")) > 1:
            n.append(x)
            if x.split("_")[0] not in names:
                names[x.split("_")[0]] = 0
            names[x.split("_")[0]] += counts[x]

    counts.update(names)
    for x in n:
        del counts[x]
    return counts

In [12]:
pd.Series(counts).to_list()

[9654, 152207, 270643, 85, 2164, 42, 2435, 1]

In [8]:
pairs = comb
pairs

[('003', '012'),
 ('003', '004'),
 ('003', '008'),
 ('003', '007'),
 ('003', '011'),
 ('003', '005'),
 ('003', '009'),
 ('012', '004'),
 ('012', '008'),
 ('012', '007'),
 ('012', '011'),
 ('012', '005'),
 ('012', '009'),
 ('004', '008'),
 ('004', '007'),
 ('004', '011'),
 ('004', '005'),
 ('004', '009'),
 ('008', '007'),
 ('008', '011'),
 ('008', '005'),
 ('008', '009'),
 ('007', '011'),
 ('007', '005'),
 ('007', '009'),
 ('011', '005'),
 ('011', '009'),
 ('005', '009')]

In [9]:
def get_overlap(db1, db2, dedup_dfs, counts, smiles_col="SMILES", small_threshold=100_000):
    """Efficient overlap detection using merge or isin depending on size."""
    df1 = dedup_dfs[db1]
    df2 = dedup_dfs[db2]

    len1 = counts[db1]
    len2 = counts[db2]
    
    # differentiate beween smaller and bigger
    if len1 < len2:
        small, big = df1, df2
        partitions = int(len1/small_threshold)
        
    else:
        small, big = df2, df1
        partitions = int(len2/small_threshold)
        
    if len1 < small_threshold or len2 < small_threshold:
        small = small.repartition(npartitions=1)
        overlap = dd.merge(big, small, on=smiles_col, how="inner")
    else:
        # Use merge for large–large comparisons
        ob = []
        meta = small._meta
        small = small.repartition(npartitions=partitions)
        for sma in small.to_delayed():
            sma = dd.from_delayed(sma, meta=meta)
            overlap = dd.merge(big, sma, on=smiles_col, how="inner")
            ob.append(overlap)
    
        overlap = dask.dataframe.concat(ob)
    return overlap

In [8]:
from itertools import islice

def batched(iterable, n):
    it = iter(iterable)
    while batch := list(islice(it, n)):
        yield batch
        
overlaps={}
for batch in batched(pairs, 3):  # run 3 at a time
    futures = []
    for db1, db2 in batch:
        overlap = dd.merge(
            dedup_dfs[db1],
            dedup_dfs[db2],
            on=smiles_col,
            how="inner"
        )
        #overlap = get_overlap(db1, db2, dedup_dfs, counts, smiles_col=smiles_col)
        futures.append(overlap)
    
    results = dask.compute(*futures)
    for (db1, db2), res in zip(batch, results):
        overlaps[f"{db1}_{db2}"] = res

In [21]:
from itertools import islice

def batched(iterable, n):
    it = iter(iterable)
    while batch := list(islice(it, n)):
        yield batch
        
overlaps={}
for batch in batched(pairs, 3):  # run 3 at a time
    futures = []
    for db1, db2 in batch:
        overlap = get_overlap(db1, db2, dedup_dfs, counts, smiles_col=smiles_col)
        futures.append(overlap)
    
    results = dask.compute(*futures)
    for (db1, db2), res in zip(batch, results):
        overlaps[f"{db1}_{db2}"] = res

In [ ]:
overlap = dedup_dfs["003"][dedup_dfs["003"]["SMILES"].isin(sma_df)]

In [14]:
meta = dedup_dfs["004"]._meta  # schema of a partition

lab = []
for part in dedup_dfs["004"].to_delayed():
    sma_df = part["SMILES"]
    overlap = dedup_dfs["003"][dedup_dfs["003"]["SMILES"].isin(sma_df)]
    lab.append(overlap)

In [3]:
db1, db2 = "a", "b"
counts = {db1: 30, db2: 50}
small_threshold = 50

In [4]:
all(i <= small_threshold for i in [counts[u] for u in [db1, db2]])

True

In [12]:
a = dask.dataframe.concat(lab).compute()
a

,SMILES
5,CC=CCC=CC(C)C
10,CC(C)=CC=CCC#N
32,C=CCOCC(=O)OC
0,CN(O)c1ccccc1
31,C#CCCn1ccnc1
...,...
13,CC=C(C)C=CCC#N
2,CCC=CC(C)(C)CC
11,CCCC=CCC1CO1
0,CC(=O)OC1CC(C)C1


In [4]:
smiles_to_dbs = defaultdict(set)
redun_counts = {}
for pair, df in overlaps.items():
    redun_counts[pair] = df.shape[0]
    db1, db2 = pair.split("_")
    for smi in df["SMILES"]:
        smiles_to_dbs[smi].update([db1, db2])

NameError: name 'overlaps' is not defined

In [22]:
smiles_overlap_df = pd.DataFrame({
    "SMILES": list(smiles_to_dbs.keys()),
    "Databases": [",".join(sorted(list(v))) for v in smiles_to_dbs.values()]
})
smiles_overlap_df

,SMILES,Databases
0,C=CCNc1ccco1,"003,004,012"
1,CC(C)OC(=O)CON,"003,004,012"
2,CCOCCOCCO,"003,004,005,007,012"
3,N#CCCOCC(N)=O,"003,004,012"
4,CCCN(C)S(C)(=O)=O,"003,004,012"
...,...,...
110508,Oc1ncc(O)c(O)n1,"005,007"
110509,Nc1cc(=O)nc(N)[nH]1,"005,007"
110510,C=C1C(=O)O[C@@H](C)[C@@H]1O,"005,007"
110511,N[C@H]1CCN[C@H]1C(=O)O,"005,007"


In [33]:
redun_counts

{'003_012': 2993,
 '003_004': 4593,
 '003_008': 3,
 '003_007': 177,
 '003_011': 4,
 '003_005': 198,
 '003_009': 1,
 '012_004': 107289,
 '012_008': 55,
 '012_007': 1534,
 '012_011': 9,
 '012_005': 1895,
 '012_009': 1,
 '004_008': 52,
 '004_007': 1352,
 '004_011': 13,
 '004_005': 1841,
 '004_009': 0,
 '008_007': 70,
 '008_011': 0,
 '008_005': 30,
 '008_009': 0,
 '007_011': 0,
 '007_005': 481,
 '007_009': 1,
 '011_005': 0,
 '011_009': 0,
 '005_009': 1}